In [1]:
import gym
import numpy as np
import random

In [2]:
env= gym.make("MountainCar-v0")

[2019-10-09 08:30:16,744] Making new env: MountainCar-v0
/home/win10/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
low = env.observation_space.low
high = env.observation_space.high


In [4]:
def create_grids(l,h,bins=(10,10)):
    grid = [np.linspace(l[dim],h[dim],bins[dim]+1)[1:-1] for dim in range(len(bins))]
    return grid


In [5]:
def discrete(sample_state, grid):
    return list(int(np.digitize(s,g)) for s, g in zip(sample_state, grid))

In [6]:
state_grid = create_grids(env.observation_space.low, env.observation_space.high, bins=(10, 10))

In [7]:
class my_agent():
    
    def __init__(self,grid,env):
            self.env = env
            self.grid = grid
            self.state_size = tuple(len(i)+1 for i in self.grid)
            print("State space size:", self.state_size)
            self.action_size = env.action_space.n
            self.gamma = 0.99
            self.alpha = 0.02
            self.epsilon = 1.0
            self.decay = 0.0001 
            self.min_epsilon = 0.01
            self.q_table = np.zeros(shape=(self.state_size + (self.action_size,)))
            print("Q table size:", self.q_table.shape)
        
    def get_state(self,state):
            return tuple(discrete(state, self.grid))
                     
                                          
    def _update_epsilon(self):
            self.epsilon -= self.decay
            self.epsilon = max(self.min_epsilon, self.epsilon)

    def act(self, state, is_new_episode):
        
#             if is_new_episode:
#                 return self.env.action_space.sample()
                
                
        
            if np.random.rand() <= self.epsilon:
                         self._update_epsilon()
                         return self.env.action_space.sample()
                         
            state =  self.get_state(state)
            action = np.argmax(self.q_table[state])
            return action             
                         
    def q_update(self,state, action, reward, next_state):
                         state =  self.get_state(state)
                         next_state = self.get_state(next_state)
                         self.q_table[state + (action,)] += self.alpha * \
                        (reward + self.gamma * max(self.q_table[next_state]) - \
                         self.q_table[state + (action,)])
                         
agent = my_agent(state_grid, env)                         

State space size: (10, 10)
Q table size: (10, 10, 3)


In [8]:
total_episode = 20_000
i = 0
for ep in range(total_episode):
                done = False
                state = env.reset()
                total_reward = 0
                action = agent.act(state,True)

                
                i +=1
                while not done:
                    
                    env.render()
                    next_state, reward, done, _ = env.step(action)
                    agent.q_update(state, action, reward, next_state)
                    total_reward += reward   
                    state = next_state    
                    action = agent.act(state, False)
                if done:
                           print("total reward of episode {} is {}".format(i,total_reward))
                           



total reward of episode 1 is -200.0
total reward of episode 2 is -200.0
total reward of episode 3 is -200.0
total reward of episode 4 is -200.0
total reward of episode 5 is -200.0
total reward of episode 6 is -200.0
total reward of episode 7 is -200.0
total reward of episode 8 is -200.0
total reward of episode 9 is -200.0
total reward of episode 10 is -200.0
total reward of episode 11 is -200.0
total reward of episode 12 is -200.0
total reward of episode 13 is -200.0
total reward of episode 14 is -200.0
total reward of episode 15 is -200.0
total reward of episode 16 is -200.0
total reward of episode 17 is -200.0
total reward of episode 18 is -200.0
total reward of episode 19 is -200.0
total reward of episode 20 is -200.0
total reward of episode 21 is -200.0
total reward of episode 22 is -200.0
total reward of episode 23 is -200.0
total reward of episode 24 is -200.0
total reward of episode 25 is -200.0
total reward of episode 26 is -200.0
total reward of episode 27 is -200.0
total rewa

total reward of episode 220 is -200.0
total reward of episode 221 is -200.0
total reward of episode 222 is -200.0
total reward of episode 223 is -200.0
total reward of episode 224 is -200.0
total reward of episode 225 is -200.0
total reward of episode 226 is -200.0
total reward of episode 227 is -200.0
total reward of episode 228 is -200.0
total reward of episode 229 is -200.0
total reward of episode 230 is -200.0
total reward of episode 231 is -200.0
total reward of episode 232 is -200.0
total reward of episode 233 is -200.0
total reward of episode 234 is -200.0
total reward of episode 235 is -200.0
total reward of episode 236 is -200.0
total reward of episode 237 is -200.0
total reward of episode 238 is -200.0
total reward of episode 239 is -200.0
total reward of episode 240 is -200.0
total reward of episode 241 is -200.0
total reward of episode 242 is -200.0
total reward of episode 243 is -200.0
total reward of episode 244 is -200.0
total reward of episode 245 is -200.0
total reward

total reward of episode 436 is -200.0
total reward of episode 437 is -200.0
total reward of episode 438 is -200.0
total reward of episode 439 is -200.0
total reward of episode 440 is -200.0
total reward of episode 441 is -200.0
total reward of episode 442 is -200.0
total reward of episode 443 is -200.0
total reward of episode 444 is -200.0
total reward of episode 445 is -200.0
total reward of episode 446 is -200.0
total reward of episode 447 is -200.0
total reward of episode 448 is -200.0
total reward of episode 449 is -200.0
total reward of episode 450 is -200.0
total reward of episode 451 is -200.0
total reward of episode 452 is -200.0
total reward of episode 453 is -200.0
total reward of episode 454 is -200.0
total reward of episode 455 is -200.0
total reward of episode 456 is -200.0
total reward of episode 457 is -200.0
total reward of episode 458 is -200.0
total reward of episode 459 is -200.0
total reward of episode 460 is -200.0
total reward of episode 461 is -200.0
total reward

total reward of episode 652 is -154.0
total reward of episode 653 is -200.0
total reward of episode 654 is -200.0
total reward of episode 655 is -165.0
total reward of episode 656 is -170.0
total reward of episode 657 is -200.0
total reward of episode 658 is -200.0
total reward of episode 659 is -200.0
total reward of episode 660 is -200.0
total reward of episode 661 is -200.0
total reward of episode 662 is -200.0
total reward of episode 663 is -192.0
total reward of episode 664 is -200.0
total reward of episode 665 is -200.0
total reward of episode 666 is -200.0
total reward of episode 667 is -200.0
total reward of episode 668 is -200.0
total reward of episode 669 is -200.0
total reward of episode 670 is -200.0
total reward of episode 671 is -200.0
total reward of episode 672 is -200.0
total reward of episode 673 is -200.0
total reward of episode 674 is -200.0
total reward of episode 675 is -200.0
total reward of episode 676 is -200.0
total reward of episode 677 is -200.0
total reward

total reward of episode 868 is -200.0
total reward of episode 869 is -200.0
total reward of episode 870 is -200.0
total reward of episode 871 is -200.0
total reward of episode 872 is -200.0
total reward of episode 873 is -200.0
total reward of episode 874 is -200.0
total reward of episode 875 is -200.0
total reward of episode 876 is -200.0
total reward of episode 877 is -200.0
total reward of episode 878 is -200.0
total reward of episode 879 is -200.0
total reward of episode 880 is -200.0
total reward of episode 881 is -200.0
total reward of episode 882 is -200.0
total reward of episode 883 is -200.0
total reward of episode 884 is -200.0
total reward of episode 885 is -184.0
total reward of episode 886 is -200.0
total reward of episode 887 is -200.0
total reward of episode 888 is -189.0
total reward of episode 889 is -164.0
total reward of episode 890 is -200.0
total reward of episode 891 is -200.0
total reward of episode 892 is -200.0
total reward of episode 893 is -200.0
total reward

ArgumentError: argument 2: <class 'TypeError'>: wrong type